# 

In [13]:
library(foreach)
library(doSNOW)
library(parallel)
library(dplyr)
library(mvtnorm)

source('~/isolines_uq/scripts/R/confidence_regions_procedure/auxiliary_scripts/distributionIsolines.R')
source('~/isolines_uq/scripts/R/confidence_regions_procedure/auxiliary_scripts/utils.R')
source('~/isolines_uq/scripts/R/confidence_regions_procedure/auxiliary_scripts/confidenceRegions.R')

save_path <- '/pscratch/sd/j/jbbutler/extreme_tubes/rv_index_unknown/'

rdist <- function(n) {
    dat <- data.frame(rmvt(n=n, sigma=matrix(c(1, 0.7, 0.7, 1), nrow=2), df=1))
    return(dat)
}

ns <- c(1000, 5000, 10000, 50000)
dist <- 'copula_tdf1_marginals_tdf1'
alphas <- c(0.05, 0.1, 0.01)
gamma <- 1/2
xi <- 1
pn <- function(n) { 5/n }

# number of cores for parallelization
n_cores <- 64
# number of simulations, i.e. number of bootstrap estimates of c_{1-alpha}
n_sims <- 100
# number of bootstrap replicates
#B <- 1000
B <- 10

## Parameters for drawing isolines to check coverage
# number of coords for isoline drawing
numCoords <- 200
lvlset_ubs <- c(50, 50)
lvlset_lbs <- c(0, 0)

# part of loop that will be parallelized: the computation of c for each simulation and evaluation of coverage
parallelizedCode <- function(ind) {

    dat <- rdist(n)
    # convert marginals to be draws from a lomax distribution (pareto starting at 0)
    # shape and scale both 1

    transform <- function(pts) {
        transformed_pts <- 1/(1-est_cdf(pts, dat[,1], gamma)) - 1
        return(transformed_pts)
    }

    inv_transform <- function(pts) {
        backtransformed_pts <- est_inv_cdf(1-(1/(1+pts)), dat, gamma)
        return(backtransformed_pts)
    }
    
    transformed_dat_X1 <- transform(dat[,1])
    transformed_dat_X2 <- transform(dat[,2])
    transformed_dat <- data.frame(X1=transformed_dat_X1, X2=transformed_dat_X2)
    
    regions <- drawExtremeRegion(transformed_dat, alphas, p, B, gamma, xi)

    # evaluate coverage
    transformed_isoline <- data.frame(X1=transform(isoline[,1]), X2=transform(isoline[,2]))
    est_survfunc <- apply(transformed_isoline, 1, blendedSurvivalFunc, dat=transformed_dat, gamma=gamma, xi=xi)
    is_covereds <- list()
    cs <- regions$c_estimates

    for (i in 1:length(alphas)) {
        alpha <- alphas[i]
        c <- cs[[as.character(alpha)]]
        print(c)
        is_covereds[as.character(alpha)] <- all((est_survfunc <= p + c) & (est_survfunc >= p - c))
    }
    regions$is_covereds <- is_covereds
    return(regions)
}

In [14]:
n <- ns[1]
p <- pn(n)

isoline <- drawBivtIsoline(numCoords=numCoords, gridUbs=lvlset_ubs, gridLbs=lvlset_lbs, prob=p, df=1)

In [15]:
res <- parallelizedCode(1)

[1] 0.002654298
[1] 0.002380011
[1] 0.002873728


In [17]:
qt(1-(5/50000), df=1)

[1] 3183.099

In [19]:
ns <- c(1000, 10000, 100000)
qt(1-(5/ns), df=1) + 100

[1]  163.6567  736.6192 6466.1977

In [ ]:
res <- readRDS('/pscratch/sd/j/jbbutler/extreme_tubes/rv_index_unknown/copula_tdf1_marginals_tdf1/p5_div_n/')